# NER Transformer Notebook Training

This is a notebook detailing the training of a transformer NER model using HuggingFace transformers.

## 1. Installs and Imports

In [ ]:
!pip install datasets transformers seqeval

In [ ]:
import os
import random
from collections import Counter, OrderedDict, defaultdict
from datetime import date

import numpy as np
import pandas as pd

# inference
import torch
import transformers
from datasets import ClassLabel, Sequence, load_dataset, load_from_disk, load_metric
from google.colab import drive
from IPython.display import HTML, display
from seqeval.metrics import accuracy_score
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    pipeline,
)

print(transformers.__version__)

In [ ]:
system = "COLAB"  # ["AWS", "COLAB"]

In [ ]:
if system == "AWS":
    fs = s3fs.S3FileSystem()
    s3_bucket = "govuk-data-infrastructure-integration"
    DATA_DIR = f"s3://{s3_bucket}/model-data/govner-data"
    for f in fs.ls(DATA_DIR):
        print(f)
    # Manage interactions with the Amazon SageMaker APIs and any other AWS services needed.
    # sagemaker session bucket -> used for uploading data, models and logs
    # sagemaker will automatically create this bucket if it not exists
    sess = sagemaker.Session()
    sagemaker_session_bucket = s3_bucket
    if sagemaker_session_bucket is None and sess is not None:
        # set to default bucket if a bucket name is not given
        sagemaker_session_bucket = sess.default_bucket()

    role = sagemaker.get_execution_role()
    sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

    print(f"sagemaker role arn: {role}")
    print(f"sagemaker bucket: {sess.default_bucket()}")
    print(f"sagemaker session region: {sess.boto_region_name}")
elif system == "COLAB":
    drive.mount("/content/gdrive")
    DATA_DIR = os.path.join(
        "/content/gdrive/Shared drives/",
        "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data",
    )
    MODEL_DIR = os.path.join(
        "/content/gdrive/Shared drives/",
        "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models",
    )
    RESULTS_DIR = os.path.join(
        "/content/gdrive/Shared drives/",
        "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Models/Metrics",
    )

In [ ]:
print("Data Folder: {}".format(DATA_DIR))
print(os.listdir(DATA_DIR)[:3])
print("Model Folder: {}".format(MODEL_DIR))
print(os.listdir(MODEL_DIR)[:3])

## 2. Load Data

Define some variables that will be useful.

In [ ]:
task = "ner"
dataset_name = "govuk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
dataset_type = "FULL"  # "FULL"

In [ ]:
if dataset_type == "SAMPLED":
    hf_data = "samp_hf_govuk_data"
    hf_data_path = f"{DATA_DIR}/{hf_data}"
    print("Data path: {}".format(hf_data_path))
elif dataset_type == "FULL":
    hf_data = "hf_govuk_data"
    hf_data_path = f"{DATA_DIR}/{hf_data}"
    print("Data path: {}".format(hf_data_path))

Load the dataset that has been saved to disk in a HuggingFace DatasetDict (Apache Arrow).

In [ ]:
datasets = load_from_disk(hf_data_path)

In [ ]:
datasets

Inspect an element

In [ ]:
datasets["train"][5]

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the features of the dataset:

In [ ]:
label_map = {
    "O": 0,
    "I-CONTACT": 1,
    "I-DATE": 2,
    "I-EVENT": 3,
    "I-FINANCE": 4,
    "I-FORM": 5,
    "I-LOC": 6,
    "I-MISC": 7,
    "I-MONEY": 8,
    "I-ORG": 9,
    "I-PER": 10,
    "I-SCHEME": 11,
    "I-STATE": 12,
}

In [ ]:
labels = [
    "O",
    "I-CONTACT",
    "I-DATE",
    "I-EVENT",
    "I-FINANCE",
    "I-FORM",
    "I-LOC",
    "I-MISC",
    "I-MONEY",
    "I-ORG",
    "I-PER",
    "I-SCHEME",
    "I-STATE",
]

In [ ]:
datasets["train"].features[f"new_label_list_id"] = Sequence(
    feature=ClassLabel(num_classes=13, names=labels, names_file=None, id=None),
    length=-1,
    id=None,
)
datasets["test"].features[f"new_label_list_id"] = Sequence(
    feature=ClassLabel(num_classes=13, names=labels, names_file=None, id=None),
    length=-1,
    id=None,
)

In [ ]:
label_list = datasets["train"].features[f"new_label_list_id"].feature.names
label_list

Show some random examples from the dataset in HTML format - this makes it easier to read than from the json.

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

## 3. Tokenise the Data

Download tokeniser that will be used to tokenise the data.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The assert keyword lets you test if a condition in your code returns True, if not, the program will raise an AssertionError.

In [ ]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

How does the tokeniser work?

In [ ]:
# observe how the tokeniser works on a string
tokenizer("Hello, this is one sentence!")

In [ ]:
# observe how the tokeniser works on a list of tokens
tokenizer(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

Try this out on example, tokens 4 from training set.

In [ ]:
example = datasets["train"][5]
print(example["text_token"])

In [ ]:
tokenized_input = tokenizer(example["text_token"], is_split_into_words=True)
print(tokenized_input)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

Print the number of token labels in the data, and the length of the tokenised input. They are different, because special tokens are added to the start and end of a list when tokenised.

In [ ]:
len(example[f"new_label_list_id"]), len(tokenized_input["input_ids"])

If we look into these examples, we can see they are added to the start and end.

In [ ]:
print(tokenized_input.word_ids())

We can align these labels, by adding '-100' where there are None.

In [ ]:
word_ids = tokenized_input.word_ids()
print(word_ids)
aligned_labels = [
    -100 if i is None else example[f"new_label_list_id"][i] for i in word_ids
]
print(aligned_labels)
print(len(aligned_labels), len(tokenized_input["input_ids"]))

We now need to tokenise each example and align the labels.


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["text_token"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"new_label_list_id"]):
        word_ids = tokenized_inputs.word_ids(
            batch_index=i
        )  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for (
            word_idx
        ) in (
            word_ids
        ):  # Set the special tokens to -100. Special tokens have a word id that is None. We set the label to -100 so they are automatically ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            elif (
                word_idx != previous_word_idx
            ):  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:  # For the other tokens in a word, we set the label to either the current label or -100, depending on the label_all_tokens flag.
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Now we can tokenise and align training examples in the datasets.

In [ ]:
tokenize_and_align_labels(datasets["train"][:5])

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

## 3. Metrics

We can use common metrics, such as those used by NER evaluation strategies - however, our training data is not organised in any of the IOB1, IOB2, IOE1, IOE2, IOBES or IO formats.

For this reason, we may need to use custom metrics. But we can try with both and assess outputs.

### 3A. 'seqeval' metrics

In [ ]:
metric = load_metric("seqeval")

In [ ]:
label_list

In [ ]:
labels = [label_list[i] for i in example[f"new_label_list_id"]]
print(labels)
metric.compute(predictions=[labels], references=[labels])

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## 4. Modelling

First, instantiate a model that will be used, **make sure it is the same as the tokeniser you are using!** Use the number of labels that are in your label list - this ensures there will be an output class for each token.

In [ ]:
id2label = {str(i): label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id
)

Define the training arguments that will dictate how the model will train.

In [ ]:
model_name = model_checkpoint.split("/")[-1]
tod_date = date.today().strftime("%d-%m-%Y")
full_model_name = (
    f"{model_name}-finetuned-{task}-{dataset_name}-{dataset_type}-{tod_date}"
)
# print(full_model_name)
# print(MODEL_DIR)
OUTPUT_PATH = f"{MODEL_DIR}/{full_model_name}"
print(OUTPUT_PATH)

args = TrainingArguments(
    output_dir=OUTPUT_PATH,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    push_to_hub=False,
)

The *Data Collator* in the trainer, automatically pads the model inputs in a batch to the length of the longest example. This bypasses the need to set a global maximum sequence length, and in practice leads to faster training since we perform fewer redundant computations on the padded tokens and attention masks.

For token classification tasks, there is a dedicated *DataCollatorForTokenClassification* which expects a list of dicts, where each dict represents a single example in the dataset.



In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
tokenizer.save_pretrained(OUTPUT_PATH)

## Get Classification Report Metrics

Use the evaluate method allows you to evaluate again on the evaluation dataset or on another dataset:


In [ ]:
trainer.evaluate()

Get the precision/recall/f1 computed for each category now that training is done. Apply the same function as before on the result of the predict method.

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results_df = pd.DataFrame(results).T

In [ ]:
results_df

In [ ]:
print(RESULTS_DIR)

print(full_model_name)

excel_path = f"{RESULTS_DIR}/{full_model_name}.xlsx"

In [ ]:
results_df.to_excel(excel_path, sheet_name=full_model_name)

## Confusion Matrices

In [ ]:
for i in tokenized_datasets["test"][1]:
    print(i, len(tokenized_datasets["test"][1][i]), tokenized_datasets["test"][1][i])

Load model in

In [ ]:
MODEL_DIR
model_name = "distilbert-base-uncased"
task = "ner"
dataset_name = "govuk"
req_date = "13-12-2021"
dataset_type = "FULL"
chkpoint = "checkpoint-73500"

In [ ]:
TRAINED_MODEL_PATH = f"{MODEL_DIR}/{model_name}-finetuned-{task}-{dataset_name}-{dataset_type}-{req_date}/{chkpoint}"
TRAINED_MODEL_PATH

In [ ]:
os.listdir(TRAINED_MODEL_PATH)

Load model and tokeniser

In [ ]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TRAINED_MODEL_PATH)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    TRAINED_MODEL_PATH, num_labels=len(label_list), id2label=id2label, label2id=label2id
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer = Trainer(
    model,
    # args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results_df = pd.DataFrame(results).T

In [ ]:
labs = list(results.keys())[:12]
labs.append("O")
labs

In [ ]:
results_df

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [ ]:
true_labels_flat = [item for sublist in true_labels for item in sublist]
true_preds_flat = [item for sublist in true_predictions for item in sublist]

CM without 'O'

In [ ]:
confmat = confusion_matrix(y_true=true_labels_flat, y_pred=true_preds_flat)

confmat_2 = [i[:-1] for i in confmat[:-1]]
confmat_2 = np.array(confmat_2)
cmplot = ConfusionMatrixDisplay(confmat_2, display_labels=labs)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
cmplot.plot(ax=ax, cmap="GnBu")

CM Without 'O'

In [ ]:
confmat = confusion_matrix(y_true=true_labels_flat, y_pred=true_preds_flat)
# confmat_2 = np.array(confmat_2)
cmplot = ConfusionMatrixDisplay(confmat, display_labels=labs)
fig, ax = plt.subplots(figsize=(15, 15))
cmplot.plot(ax=ax, cmap="GnBu")